In [1]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
from datetime import datetime as dt


from control_mongo import to_mongo
# to_mongo.insert_data(data, dest)

In [2]:
data = requests.get("http://ncov.mohw.go.kr/bdBoardList_Real.do?brdId=1&brdGubun=11&ncvContSeq=&contSeq=&board_id=&gubun=")
corona_data = pd.read_html(data.content, encoding="utf8")

# 해외유입 환자현황 데이터 세팅
k = corona_data[0].loc[:,['구분.1', '신규', '누계']]
data_set = {}
data_sub_set = {}
for idx in range(len(k)):
    # pandas 의 int 형식의 값은 numpy.int 로 몽고에 insert하게되면 타입 에러 발생 --> item() 으로 대체
    
    data_sub_set[k.iloc[idx]['구분.1']] = {"신규": k.iloc[idx]['신규'].item(), "누계" : k.loc[idx]['누계'].item()}
#     print("국가 : {}, 수량 : {}".format(k.iloc[idx]['구분.1'], k.iloc[idx]["신규"]))
data_set[dt.date(dt.now()).isoformat()] = data_sub_set

# print(data_set)

In [97]:
# Data insert to mongo
mg = to_mongo()
mg.insert_data(data_set, 'corona_data')

![img](img/img.png)

In [96]:
# 해외유입 환자현황 [ 구분 / 구분.1 / 신규 / 누계 / (비율) ]
# corona_data[0]

# 누적 검사현황 [ 격리중 / 격리해제 / 사망/ 소계 / 결과음성 / 소계 / 검사중 / 함계]
# corona_data[1]

# 누적 확진자 수 [ 날짜 / 누적 확진환자수 / 일일 확진환자수]
corona_data[2:5]

# 누적 격리해제 [ 날짜 / 누적 격리해제 / 일일 격리해제 ]
# corona_data[3]

# 성별 확진자 사망자 치명률 [ 구분 / 확진자(%) / 사망자(%) / 치명률(%)]
# corona_data[4]

# 연령대 별 확진자 [ 구분 / 확진자(%) / 사망자(%) / 치명률(%) ]
# corona_data[5]

[         날짜  누적 확진환자수  일일 확진환자수
 0  20200813     14770        56
 1  20200814     14873       103
 2  20200815     15039       166
 3  20200816     15318       279
 4  20200817     15515       197
 5  20200818     15761       246
 6  20200819     16058       297,
          날짜  누적 격리해제  일일 격리해제
 0  20200813    13817       31
 1  20200814    13863       46
 2  20200815    13901       38
 3  20200816    13910        9
 4  20200817    13917        7
 5  20200818    13934       17
 6  20200819    14006       72,
    구분         확진자(%)       사망자(%)  치명률(%)
 0  남성  7,251 (45.16)  163 (53.27)    2.25
 1  여성  8,807 (54.84)  143 (46.73)    1.62]

In [5]:
# 누적 확진환자
# bs_data.find(class_="ca_value").text

# 전일대비 변화량 (소계)
# bs_data.find_all(class_="inner_value")[0]

# 전일대비 해외유입량
# bs_data.find_all(class_="inner_value")[1]

# 전일대비 국내발생량
# bs_data.find_all(class_="inner_value")[2]

# 누적 격리해제
# bs_data.find_all(class_="ca_value")[2].text

# 전일대비 격리해제
# bs_data.find_all(class_="txt_ntc")[0].text

# 누적 격리중
# bs_data.find_all(class_="ca_value")[4].text

# 전일대비 격리중
# bs_data.find_all(class_="txt_ntc")[1].text

# 누적 사망
# bs_data.find_all(class_="ca_value")[6].text

# 전일대비 사망
# bs_data.find_all(class_="txt_ntc")[2].text
